# DEPLOY THE EXAMPLE MICROSERVICES

## Node js deployment


In [2]:
!cat ../ecsdemo-nodejs/kubernetes/deployment.yaml

apiVersion: apps/v1
kind: Deployment
metadata:
  name: ecsdemo-nodejs
  labels:
    app: ecsdemo-nodejs
  namespace: default
spec:
  replicas: 1
  selector:
    matchLabels:
      app: ecsdemo-nodejs
  strategy:
    rollingUpdate:
      maxSurge: 25%
      maxUnavailable: 25%
    type: RollingUpdate
  template:
    metadata:
      labels:
        app: ecsdemo-nodejs
    spec:
      containers:
      - image: brentley/ecsdemo-nodejs:latest
        imagePullPolicy: Always
        name: ecsdemo-nodejs
        ports:
        - containerPort: 3000
          protocol: TCP


In [3]:
!cat ../ecsdemo-nodejs/kubernetes/service.yaml

apiVersion: v1
kind: Service
metadata:
  name: ecsdemo-nodejs
spec:
  selector:
    app: ecsdemo-nodejs
  ports:
   -  protocol: TCP
      port: 80
      targetPort: 3000



In [4]:
!kubectl apply -f ../ecsdemo-nodejs/kubernetes/deployment.yaml

deployment.apps/ecsdemo-nodejs created


In [5]:
!kubectl apply -f ../ecsdemo-nodejs/kubernetes/service.yaml

service/ecsdemo-nodejs created


<img src="deployed.png"> 

<img src="services.png"> 

Get deployment status

In [6]:
!kubectl get deployment ecsdemo-nodejs

NAME             READY   UP-TO-DATE   AVAILABLE   AGE
ecsdemo-nodejs   1/1     1            1           2m34s


## DEPLOY CRYSTAL BACKEND API

In [7]:
!cat ../ecsdemo-crystal/kubernetes/deployment.yaml

apiVersion: apps/v1
kind: Deployment
metadata:
  name: ecsdemo-crystal
  labels:
    app: ecsdemo-crystal
  namespace: default
spec:
  replicas: 1
  selector:
    matchLabels:
      app: ecsdemo-crystal
  strategy:
    rollingUpdate:
      maxSurge: 25%
      maxUnavailable: 25%
    type: RollingUpdate
  template:
    metadata:
      labels:
        app: ecsdemo-crystal
    spec:
      containers:
      - image: brentley/ecsdemo-crystal:latest
        imagePullPolicy: Always
        name: ecsdemo-crystal
        ports:
        - containerPort: 3000
          protocol: TCP


In [8]:
!cat ../ecsdemo-crystal/kubernetes/service.yaml

apiVersion: v1
kind: Service
metadata:
  name: ecsdemo-crystal
spec:
  selector:
    app: ecsdemo-crystal
  ports:
   -  protocol: TCP
      port: 80
      targetPort: 3000



In [9]:
!kubectl apply -f ../ecsdemo-crystal/kubernetes/deployment.yaml

deployment.apps/ecsdemo-crystal created


In [10]:
!kubectl apply -f ../ecsdemo-crystal/kubernetes/service.yaml

service/ecsdemo-crystal created


In [11]:
!kubectl get deployment ecsdemo-crystal

NAME              READY   UP-TO-DATE   AVAILABLE   AGE
ecsdemo-crystal   1/1     1            1           26s


Check that we can make a load balancer!

In [12]:
!aws iam get-role --role-name "AWSServiceRoleForElasticLoadBalancing" || aws iam create-service-linked-role --aws-service-name "elasticloadbalancing.amazonaws.com"


An error occurred (NoSuchEntity) when calling the GetRole operation: The role with name AWSServiceRoleForElasticLoadBalancing cannot be found.
{
    "Role": {
        "Path": "/aws-service-role/elasticloadbalancing.amazonaws.com/",
        "RoleName": "AWSServiceRoleForElasticLoadBalancing",
        "RoleId": "AROA5U2PS7UL7YS3ADTTO",
        "Arn": "arn:aws:iam::938080402711:role/aws-service-role/elasticloadbalancing.amazonaws.com/AWSServiceRoleForElasticLoadBalancing",
        "CreateDate": "2020-04-09T11:50:48Z",
        "AssumeRolePolicyDocument": {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Action": [
                        "sts:AssumeRole"
                    ],
                    "Effect": "Allow",
                    "Principal": {
                        "Service": [
                            "elasticloadbalancing.amazonaws.com"
                        ]
                    }
                }
            ]
      

## Frontend

In [13]:
!cat ../ecsdemo-frontend/kubernetes/deployment.yaml

apiVersion: apps/v1
kind: Deployment
metadata:
  name: ecsdemo-frontend
  labels:
    app: ecsdemo-frontend
  namespace: default
spec:
  replicas: 1
  selector:
    matchLabels:
      app: ecsdemo-frontend
  strategy:
    rollingUpdate:
      maxSurge: 25%
      maxUnavailable: 25%
    type: RollingUpdate
  template:
    metadata:
      labels:
        app: ecsdemo-frontend
    spec:
      containers:
      - image: brentley/ecsdemo-frontend:latest
        imagePullPolicy: Always
        name: ecsdemo-frontend
        ports:
        - containerPort: 3000
          protocol: TCP
        env:
        - name: CRYSTAL_URL
          value: "http://ecsdemo-crystal.default.svc.cluster.local/crystal"
        - name: NODEJS_URL
          value: "http://ecsdemo-nodejs.default.svc.cluster.local/"


In [14]:
!cat ../ecsdemo-frontend/kubernetes/service.yaml

apiVersion: v1
kind: Service
metadata:
  name: ecsdemo-frontend
spec:
  selector:
    app: ecsdemo-frontend
  type: LoadBalancer
  ports:
   -  protocol: TCP
      port: 80
      targetPort: 3000



In [15]:
!kubectl apply -f ../ecsdemo-frontend/kubernetes/deployment.yaml

deployment.apps/ecsdemo-frontend created


In [16]:
!kubectl apply -f ../ecsdemo-frontend/kubernetes/service.yaml

service/ecsdemo-frontend created


In [17]:
!kubectl get deployment ecsdemo-frontend

NAME               READY   UP-TO-DATE   AVAILABLE   AGE
ecsdemo-frontend   1/1     1            1           38s


- need to find ELB address

In [19]:
!kubectl get service ecsdemo-frontend -o wide

NAME               TYPE           CLUSTER-IP      EXTERNAL-IP                                                               PORT(S)        AGE    SELECTOR
ecsdemo-frontend   LoadBalancer   10.100.142.58   aa2934e4f7a5811ea842d12800186a62-2055527445.us-east-1.elb.amazonaws.com   80:32701/TCP   2m1s   app=ecsdemo-frontend


In [22]:
!curl -m3 -v $(kubectl get service ecsdemo-frontend -o json | jq -r '.status.loadBalancer.ingress[].hostname')

* Rebuilt URL to: aa2934e4f7a5811ea842d12800186a62-2055527445.us-east-1.elb.amazonaws.com/
*   Trying 34.206.201.161...
* TCP_NODELAY set
* Connected to aa2934e4f7a5811ea842d12800186a62-2055527445.us-east-1.elb.amazonaws.com (34.206.201.161) port 80 (#0)
> GET / HTTP/1.1
> Host: aa2934e4f7a5811ea842d12800186a62-2055527445.us-east-1.elb.amazonaws.com
> User-Agent: curl/7.54.0
> Accept: */*
> 
< HTTP/1.1 200 OK
< X-Frame-Options: SAMEORIGIN
< X-XSS-Protection: 1; mode=block
< X-Content-Type-Options: nosniff
< Content-Type: text/html; charset=utf-8
< ETag: W/"d84ca474d808a857f71dc2114e16ada5"
< Cache-Control: max-age=0, private, must-revalidate
< X-Request-Id: b64a3d66-46b3-4121-8141-5b5e1ea390bd
< X-Runtime: 0.076899
< Connection: close
< Server: thin
< 
<!DOCTYPE html>
<html>
<head>
  <title>ExampleApp</title>
  <script>
    function load() {
      setTimeout(function() {location=''}, 50)
    }
    window.onload = load;
  </script>
</head>
<body>

<style>
body {
    background-color: #6

## Scale the backend

In [23]:
!kubectl get deployments

NAME               READY   UP-TO-DATE   AVAILABLE   AGE
ecsdemo-crystal    1/1     1            1           8m24s
ecsdemo-frontend   1/1     1            1           4m25s
ecsdemo-nodejs     1/1     1            1           12m


In [24]:
!kubectl scale deployment ecsdemo-nodejs --replicas=3

deployment.extensions/ecsdemo-nodejs scaled


In [26]:
!kubectl scale deployment ecsdemo-crystal --replicas=3

deployment.extensions/ecsdemo-crystal scaled


In [27]:
!kubectl get deployments

NAME               READY   UP-TO-DATE   AVAILABLE   AGE
ecsdemo-crystal    3/3     3            3           9m2s
ecsdemo-frontend   1/1     1            1           5m3s
ecsdemo-nodejs     3/3     3            3           13m


## Scale frontend

In [28]:
!kubectl scale deployment ecsdemo-frontend --replicas=3

deployment.extensions/ecsdemo-frontend scaled


In [31]:
!kubectl get deployments

NAME               READY   UP-TO-DATE   AVAILABLE   AGE
ecsdemo-crystal    3/3     3            3           10m
ecsdemo-frontend   3/3     3            3           6m23s
ecsdemo-nodejs     3/3     3            3           14m


In [33]:
from IPython.display import Video

Video("lb-in-action.mp4",width=400,height=400)

## Clean everything up

In [34]:
!kubectl delete -f ../ecsdemo-nodejs/kubernetes/deployment.yaml
!kubectl delete -f ../ecsdemo-nodejs/kubernetes/service.yaml

deployment.apps "ecsdemo-nodejs" deleted
service "ecsdemo-nodejs" deleted


In [35]:
!kubectl delete -f ../ecsdemo-crystal/kubernetes/deployment.yaml
!kubectl delete -f ../ecsdemo-crystal/kubernetes/service.yaml

deployment.apps "ecsdemo-crystal" deleted
service "ecsdemo-crystal" deleted


In [36]:
!kubectl delete -f ../ecsdemo-frontend/kubernetes/deployment.yaml
!kubectl delete -f ../ecsdemo-frontend/kubernetes/service.yaml

deployment.apps "ecsdemo-frontend" deleted
service "ecsdemo-frontend" deleted
